# 1. Install and Import Libraries

In [0]:
%sh
pip install --quiet tqdm huggingface_hub aiohttp nest_asyncio aiolimiter glob sqlite3

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob

[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [0]:
import argparse
import asyncio
from dataclasses import dataclass
import datetime as dt
import gzip
from itertools import islice
import json
import logging
from logging.handlers import TimedRotatingFileHandler
import nest_asyncio
import os
import random
import re
import shutil
import sys
import tempfile
import time
from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
from urllib.parse import urlparse

import aiohttp
from aiolimiter import AsyncLimiter
from bs4 import BeautifulSoup
import glob
from huggingface_hub import HfApi
import sqlite3
import numpy as np
import pandas as pd
from pandas.api.types import is_object_dtype
import requests
from tqdm import tqdm

nest_asyncio.apply()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 2. Declare Constants

In [0]:
PROGRESS_PATH = ".hf_progress_ids.txt"
ENCODING_TYPE = 'utf-8-sig'
BASE = "https://huggingface.co"
HF_READ_TOKEN = "YOUR_READ_TOKEN"
HF_WRITE_TOKEN = "YOUR_WRITE_TOKEN"
META_CSV = "hf_metadata.csv"
FILES_CSV = "hf_files.csv"
COMMU_CSV = "hf_community.csv"

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 3. Setup Logging

In [0]:
def setup_logging(log_to_file=False):
    fmt = "[%(asctime)s][%(levelname)s][%(name)s] %(message)s"

    handlers = []

    # 노트북 셀에 보이도록 stdout으로!
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter(fmt))
    console_handler.setLevel(logging.INFO)
    handlers.append(console_handler)

    if log_to_file:
        log_dir = "/dbfs/logs"  # /logs 대신 DBFS 경로 권장
        os.makedirs(log_dir, exist_ok=True)
        file_handler = TimedRotatingFileHandler(
            filename=os.path.join(log_dir, "project.log"),
            when="midnight",
            interval=1,
            backupCount=7,
            encoding=ENCODING_TYPE
        )
        file_handler.setFormatter(logging.Formatter(fmt))
        file_handler.setLevel(logging.INFO)
        handlers.append(file_handler)

    # Databricks는 이미 로깅이 초기화되어 있을 수 있음 → 강제 재설정
    logging.basicConfig(level=logging.INFO, handlers=handlers, force=True)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 4. User Defined Function: Utility

## 4-1. make_hashable

In [0]:
def make_hashable(data: Any) -> Union[tuple, str, None]:
    if isinstance(data, (dict, list)):
        return json.dumps(data, ensure_ascii=False)
    return data

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-2. jprint

In [0]:
def jprint(data: Any):
    print(json.dumps(data, ensure_ascii=False, indent=4))

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-3. batched

In [0]:
def batched(iterable, n):
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            break
        yield batch

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-4. ensure_schema

### 1. Metadata

In [0]:
META_SCHEMA: dict[str, dict[str, str | int | float | None]] = {
    "id":                   {"dtype": "string", "default": ""},       # NOT NULL
    "hexid":                {"dtype": "string", "default": None},
    "sha":                  {"dtype": "string", "default": None},
    "author":               {"dtype": "string", "default": None},
    "private":              {"dtype": "boolean","default": None},
    "disabled":             {"dtype": "boolean","default": None},
    "gated":                {"dtype": "string", "default": None},
    "pipeline_tag":         {"dtype": "string", "default": None},
    "library_name":         {"dtype": "string", "default": None},
    "tags":                 {"dtype": "string", "default": None},     # JSON 문자열
    "likes":                {"dtype": "Float64", "default": None},
    "downloads":            {"dtype": "Float64", "default": None},
    "trending_score":       {"dtype": "Float64",  "default": 0.0},      # NOT NULL
    "created_at":           {"dtype": "timestamp with timezone", "default": None},     # ISO8601 권장
    "last_modified":        {"dtype": "timestamp with timezone", "default": None},     # ISO8601 권장
    "config":               {"dtype": "string", "default": None},     # JSON 문자열
    "card_data":            {"dtype": "string", "default": None},     # JSON 문자열
    "safetensors":          {"dtype": "string", "default": None},     # JSON 문자열
    "spaces":               {"dtype": "string", "default": None},     # JSON 배열 문자열
    "transformers_info":    {"dtype": "string", "default": None},     # JSON 문자열
    "used_storage":         {"dtype": "Float64",  "default": None},
    "readme":               {"dtype": "string", "default": None},

    "model_index":          {"dtype": "string", "default": None},     # JSON 배열 문자열
    "gguf":                 {"dtype": "string", "default": None},
    "inference":            {"dtype": "string", "default": None},
    "mask_token":           {"dtype": "string", "default": None},
    "widget_data":          {"dtype": "string", "default": None},
    "security_repo_status": {"dtype": "string", "default": None},
}

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 2. Files

In [0]:
FILES_SCHEMA: dict[str, dict[str, str | int | float | None]] = {
    "model_id":        {"dtype": "string", "default": ""},     # NOT NULL
    "type":            {"dtype": "string", "default": ""},     # NOT NULL
    "oid":             {"dtype": "string", "default": ""},     # NOT NULL (CHAR(40))
    "size":            {"dtype": "Int64",  "default": 0},      # NOT NULL
    "path":            {"dtype": "string", "default": None},
    "lfs_oid":         {"dtype": "string", "default": None},
    "lfs_size":        {"dtype": "Int64",  "default": None},
    "lfs_pointerSize": {"dtype": "Int64",  "default": None},
    "xetHash":         {"dtype": "string", "default": None},
}

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 3. Community

In [0]:
COMMU_SCHEMA: dict[str, dict[str, str | int | float | None]] = {
    "model_id":                     {"dtype": "string",  "default": ""},      # NOT NULL
    "num":                          {"dtype": "Int64",   "default": 0},       # NOT NULL (PK*)
    "title":                        {"dtype": "string",  "default": None},
    "status":                       {"dtype": "string",  "default": None},
    "createdAt":                    {"dtype": "string",  "default": None},    # ISO8601 권장
    "isPullRequest":                {"dtype": "boolean", "default": None},
    "numComments":                  {"dtype": "Int64",   "default": None},
    "topReactions":                 {"dtype": "string",  "default": None},    # JSON 문자열
    "numReactionUsers":             {"dtype": "Int64",   "default": None},
    "pinned":                       {"dtype": "boolean", "default": None},
    "author__id":                   {"dtype": "string",  "default": None},
    "author_avatarUrl":             {"dtype": "string",  "default": None},
    "author_fullname":              {"dtype": "string",  "default": None},
    "author_name":                  {"dtype": "string",  "default": None},
    "author_type":                  {"dtype": "string",  "default": None},
    "author_isPro":                 {"dtype": "boolean", "default": False},   # NOT NULL
    "author_isHf":                  {"dtype": "boolean", "default": False},   # NOT NULL
    "author_isHfAdmin":             {"dtype": "boolean", "default": False},   # NOT NULL
    "author_isMod":                 {"dtype": "boolean", "default": False},   # NOT NULL
    "repo_name":                    {"dtype": "string",  "default": ""},      # NOT NULL (PK*)
    "repo_type":                    {"dtype": "string",  "default": None},
    "repoOwner_name":               {"dtype": "string",  "default": None},
    "repoOwner_isParticipating":    {"dtype": "boolean", "default": None},
    "repoOwner_type":               {"dtype": "string",  "default": None},
    "repoOwner_isDiscussionAuthor": {"dtype": "boolean", "default": None},
    "author_followerCount":         {"dtype": "float",   "default": None},
}

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 4. Functions

In [0]:
# 안전 직렬화 헬퍼들
def _is_scalar(x) -> bool:
    return np.isscalar(x) or isinstance(x, (str, bytes)) or x is None


def _to_json_string(x):
    try:
        return json.dumps(x, ensure_ascii=False)
    except Exception:
        return str(x)


def _cast_series(s: pd.Series, dtype: str) -> pd.Series:
    if dtype == "string":
        return s.map(
            lambda v: (
                None if _is_scalar(v) and pd.isna(v)
                else (_to_json_string(v) if not _is_scalar(v) else str(v))
            )
        ).astype("string")

    if dtype == "boolean":
        return s.map(
            lambda v: (
                None if _is_scalar(v) and pd.isna(v)
                else (bool(v) if _is_scalar(v) else None)
            )
        ).astype("boolean")

    if dtype == "Int64":
        return s.map(lambda v: (v if _is_scalar(v) else None)).pipe(
            pd.to_numeric, errors="coerce"
        ).astype("Int64")

    if dtype == "float":
        return s.map(lambda v: (v if _is_scalar(v) else None)).pipe(
            pd.to_numeric, errors="coerce"
        )

    # object-like fallback
    return s.map(
        lambda v: (
            None if _is_scalar(v) and pd.isna(v)
            else (_to_json_string(v) if not _is_scalar(v) else v)
        )
    )

def ensure_schema(df: pd.DataFrame, schema: dict[str, dict[str, str]]) -> pd.DataFrame:
    # 누락 컬럼 채우고 순서 재정렬
    for col, spec in schema.items():
        if col not in df.columns:
            df[col] = spec["default"]
    df = df.reindex(columns=list(schema.keys()))

    # dtype 보정
    for col, spec in schema.items():
        dtype = spec["dtype"]
        try:
            df[col] = _cast_series(df[col], dtype)
        except Exception:
            df[col] = df[col].map(
                lambda v: None if _is_scalar(v) and pd.isna(v)
                else (_to_json_string(v) if not _is_scalar(v) else str(v))
            ).astype("string")
    return df

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-5. _load_progress

In [0]:
def _load_progress() -> set[str]:
    if not os.path.exists(PROGRESS_PATH):
        return set()
    try:
        with open(PROGRESS_PATH, "r", encoding=ENCODING_TYPE) as f:
            data = json.load(f)
            return set(data if isinstance(data, list) else [])
    except Exception:
        return set()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-6. _save_progress

In [0]:
def _save_progress(done_ids: set[str]) -> None:
    tmp = PROGRESS_PATH + ".tmp"
    with open(tmp, "w", encoding=ENCODING_TYPE) as f:
        json.dump(sorted(list(done_ids)), f, ensure_ascii=False)
    os.replace(tmp, PROGRESS_PATH)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-7. _append_csv

In [0]:
def _append_csv(df: pd.DataFrame, path: str, schema: dict, key_cols: Optional[list[str]] = None) -> None:
    # 스키마 강제
    df = ensure_schema(df, schema)

    # 기존 파일 로드 후 동일 스키마 강제
    if os.path.exists(path):
        base = pd.read_csv(path, encoding=ENCODING_TYPE, dtype="string", keep_default_na=True, na_values=["", "NA", "NaN"])
        base = ensure_schema(base, schema)
        merged = pd.concat([base, df], ignore_index=True)
        if key_cols:
            merged = merged.drop_duplicates(key_cols, keep="first").reset_index(drop=True)
    else:
        merged = df

    # 안전 저장(원자적 교체)
    # tmp_out = path + ".tmp"
    merged.to_csv(path, encoding=ENCODING_TYPE, index=False, sep=",", escapechar="\\", lineterminator="\n")
    # os.replace(tmp_out, path)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 4-8. Other Async

In [0]:
def _build_connector():
    return aiohttp.TCPConnector(
        limit=0,                # 실제 동시성은 세마포어/리미터가 관리
        limit_per_host=0,
        ttl_dns_cache=300,
        enable_cleanup_closed=True,
        ssl=False,              # 환경에 따라 True로 바꿔도 됨
        force_close=False,
        use_dns_cache=True,
        keepalive_timeout=20,
    )

def _build_timeout(total_sec: float):
    # total만 걸면 충분. 필요시 connect/read 조절 가능
    return aiohttp.ClientTimeout(total=total_sec)

class TransientHTTPError(Exception):
    pass

# 공통 재시도 래퍼 (HTTP 5xx/429/타임아웃/클라이언트 오류 일시성 가정)
async def _retry(coro_factory, retries=3, base=0.4, max_sleep=5.0):
    for attempt in range(retries + 1):
        try:
            return await coro_factory()
        except Exception as e:
            msg = str(e)
            is_session_closed = "Session is closed" in msg
            is_transient = isinstance(e, (aiohttp.ClientError, asyncio.TimeoutError)) \
                           or "429" in msg or " 5" in msg[:2] or "Temporary" in msg
            if is_session_closed:
                # 상위에서 다시 큐에 넣거나 스킵하도록 None 반환
                raise RuntimeError("Session is closed") from e
            if attempt < retries and is_transient:
                # 지수 백오프 + 지터
                sleep = min(max_sleep, base * (2 ** attempt)) + random.random() * 0.2
                await asyncio.sleep(sleep)
                continue
            raise

# 리미터 앞에서 대기 -> 스파이크 없이 지속 RPS 유지
async def _limited_call(limiter: AsyncLimiter, fn, *args, **kwargs):
    async with limiter:
        return await fn(*args, **kwargs)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 5. Sync Functions

## 5-1. Build Session

In [0]:
def build_session() -> requests.Session:
    session = requests.Session()
    session.headers.update({
        "User-Agent": "hf-text-scraper/1.0 (+https://github.com/)",
        "Accept": "application/json, text/plain, */*",
    })
    if HF_READ_TOKEN:
        session.headers.update({"Authorization": f"Bearer {HF_READ_TOKEN}"})
    return session

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 5-2. Get Model Metadata

In [0]:
def get_model_meta(session: requests.Session, repo_id: str) -> dict[str, Any]:
    # Model Card
    url = f"{BASE}/api/models/{repo_id}"
    response = session.get(url, timeout=30)
    response.raise_for_status()
    return response.json()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 5-3. Get README Markdown

In [0]:
def get_readme_markdown(session: requests.Session, repo_id: str, revision: str = "main") -> str:
    # Files > README.md
    url = f"{BASE}/{repo_id}/resolve/{revision}/README.md"
    response = session.get(url, timeout=30)
    if response.status_code == 200 and response.text:
        return response.text
    
    # if fail
    url = f"{BASE}/api/models/{repo_id}/readme?revision={revision}"
    response = session.get(url, timeout=30)
    if response.status_code == 200:
        try:
            json_ = response.json()
            for key in ("text", "content", "raw", "markdown"):
                if key in json_ and isinstance(json_[key], str):
                    return json_[key]
        except Exception:
            pass
    
    return ""

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 5-4. Get Files Tree

In [0]:
def get_files_tree(
    session: requests.Session,
    repo_id: str,
    revision: str = "main",
    limit: int = 1000
) -> list[dict[str, Any]]:
    """
    커서 기반 페이지네이션으로 전체 파일 트리 수집하는 함수
    각 item 예시
    {
        "type": "blob"|"directory"|"lfs",
        "oid": "...",
        "size": 12345,
        "path": "config.json",
        "lfs": {...}  # 있을 수 있음
    }
    """
    files_tree = []
    cursor = None
    while True:
        # Files: endpoint 설정하기
        url = f"{BASE}/api/models/{repo_id}/tree/{revision}?limit={limit}"
        if cursor:
            url += f"&cursor={cursor}"
        
        # 응답 코드 확인하기 및 데이터 가져오기
        response = session.get(url, timeout=30)
        response.raise_for_status()
        json_ = response.json()
        items = json_.get("tree", []) if isinstance(json_, dict) else json_
        files_tree.extend(items)
        
        # 다음 페이지네이션 탐색
        cursor = (
            (json_.get("cursor") or json_.get("nextCursor"))
            if isinstance(json_, dict) else None
        )
        # 없다면 종료
        if not cursor or not items:
            break
        time.sleep(0.2)

    return files_tree

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 5-5. Get Community

In [0]:
def get_community(session: requests.Session, repo_id: str, limit: int = 100) -> list[dict[str, Any]]:
    """
    커서 기반 페이지네이션으로 Community(Discussions) 목록을 모두 긁어오는 함수
    가져오는 정보로는 제목, 작성자, 상태, 댓글 수 등
    """
    all_discussions = []
    cursor = None
    while True:
        try:
            # Community: endpoint 설정하기
            url = f"{BASE}/api/models/{repo_id}/discussions?limit={limit}"
            if cursor:
                url += f"&cursor={cursor}"
            
            # 응답 코드 확인하기
            response = session.get(url, timeout=30)
            if response.status_code == 404:
                # discussions 기능이 비활성/없는 경우
                break
            
            # 데이터 가져오기
            response.raise_for_status()
            json_ = response.json()
            items = json_.get("discussions") if isinstance(json_, dict) else json_
            all_discussions.extend(items)
            
            # 다음 페이지네이션 탐색
            cursor = (
                json_.get("cursor") or json_.get("nextCursor")
                if isinstance(json_, dict) else json_
            )
            # 없다면 종료
            if not cursor or not items:
                break
            time.sleep(0.2)
        
        except Exception as e:
            return e
        
    return all_discussions

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 6. Async Functions

## 6-1. Async Class Setup

In [0]:
@dataclass
class AsyncHTTPConfig:
    timeout_sec: int = 30
    max_retries: int = 5
    backoff_base: float = 0.5
    jitter: float = 0.2

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 6-2. (inner func) fetch_json

In [0]:
async def _fetch_json(session: aiohttp.ClientSession, url: str, cfg: AsyncHTTPConfig) -> Any:
    for attempt in range(cfg.max_retries):
        try:
            async with session.get(url, timeout=cfg.timeout_sec) as resp:
                if resp.status in (200, 201):
                    ctype = resp.headers.get("Content-Type", "")
                    if "json" in ctype:
                        return await resp.json()
                    
                    # await resp.text()
                    data = await resp.read()                    
                    try:
                        return data.decode("utf-8")
                    except UnicodeDecodeError:
                        return None
                
                if resp.status in (404, 410):
                    return None
                if resp.status in (429, 500, 502, 503, 504):
                    # retryable
                    pass
                else:
                    # unexpected
                    text = await resp.text()
                    # raise RuntimeError(f"HTTP {resp.status}: {text[:200]}")
                    return None
        except Exception as e:
            if attempt == cfg.max_retries - 1:
                raise
        
        # backoff
        await asyncio.sleep(cfg.backoff_base * (2 ** attempt) + random.random() * cfg.jitter)
    return None

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 6-3. (Async) Get Model Metadata

In [0]:
async def get_model_meta_async(
    session: aiohttp.ClientSession,
    repo_id: str,
    cfg: AsyncHTTPConfig
) -> dict[str, Any]:
    url = f"{BASE}/api/models/{repo_id}"
    data = await _fetch_json(session, url, cfg)
    return data if isinstance(data, dict) else None

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 6-4. (Async) Get README Markdown

In [0]:
async def get_readme_markdown_async(
    session: aiohttp.ClientSession,
    repo_id: str,
    cfg: AsyncHTTPConfig,
    revision: str = "main"
) -> str:
    url = f"{BASE}/{repo_id}/resolve/{revision}/README.md"
    data = await _fetch_json(session, url, cfg)
    if isinstance(data, str) and data:
        return data
    
    # fallback
    url = f"{BASE}/api/models/{repo_id}/readme?revision={revision}"
    data = await _fetch_json(session, url, cfg)
    if isinstance(data, dict):
        for key in ("text", "content", "raw", "markdown"):
            val = data.get(key)
            if isinstance(val, str):
                return val
    return ""

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 6-5. (Async) Get Files Tree

In [0]:
async def get_files_tree_async(
    session: aiohttp.ClientSession,
    repo_id: str,
    cfg: AsyncHTTPConfig,
    revision: str = "main",
    limit: int = 1000
) -> list[dict[str, Any]]:
    files = []
    cursor = None
    while True:
        url = f"{BASE}/api/models/{repo_id}/tree/{revision}?limit={limit}"
        if cursor:
            url += f"&cursor={cursor}"
        
        data = await _fetch_json(session, url, cfg)
        if data is None:
            break
        
        items = data.get("tree", []) if isinstance(data, dict) else data
        if not items:
            break
        
        files.extend(items)
        cursor = (
            (data.get("cursor") or data.get("nextCursor"))
            if isinstance(data, dict) else None
        )
        if not cursor:
            break
        await asyncio.sleep(0.05)
    return files

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 6-6. (Async) Get Community

In [0]:
async def get_community_async(
    session: aiohttp.ClientSession,
    repo_id: str,
    cfg: AsyncHTTPConfig,
    limit: int = 100
) -> list[dict[str, Any]]:
    results = []
    cursor = None
    while True:
        url = f"{BASE}/api/models/{repo_id}/discussions?limit={limit}"
        if cursor:
            url += f"&cursor={cursor}"
        
        data = await _fetch_json(session, url, cfg)
        if data is None:
            break
        
        items = data.get("discussions") if isinstance(data, dict) else data
        if not items:
            break
        
        results.extend(items)
        cursor = (
            data.get("cursor") or data.get("nextCursor")
            if isinstance(data, dict) else data
        )
        if not cursor:
            break
        await asyncio.sleep(0.05)
    return results


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 7. Huggingface Class

In [0]:
class HuggingFace:
    def __init__(self) -> None:
        self.api = HfApi()
        self.logger = logging.getLogger(self.__class__.__name__)
        self.session = build_session()
        self.base_url = "https://huggingface.co/"
        self.api_schema = [
            "id",                   # 모델 전체 ID, author/model_name의 형태
            "gguf",                 # GGUF 포맷 변환 여부 (LLaMA 등 양자화 모델 관련 정보)
            "inference",            # Inference API 지원 여부
            "mask_token",           # BERT류 모델에서 [MASK] 토큰 같은 마스킹 토큰 값
            "widget_data",          # 모델 위젯(사이트 우측 패널)에서 보여주는 예시 입출력
            "trending_score",       # 인기도 점수(내부 점수라 모름)
            "security_repo_status", # 보안 관련 검토 상태
            # "downloads_all_time",   # 전체 기간 다운로드 횟수(제공 안 하는 경우 다수 존재)
        ]
        self.req_schema = [
            "_id",                  # hex 코드로 들어오는 id값    
            "id",                   # 모델 전체 ID, author/model_name의 형태
            "private",              # 모델 비공개 여부
            "pipeline_tag",         # 모델의 대표 태스크
            "library_name",         # 모델이 주로 사용하는 라이브러리
            "tags",                 # 모델에 붙은 태그 리스트
            "downloads",            # 최근 1달 다운로드 횟수
            "likes",                # 좋아요 수
            "modelId",              # id와 동일
            "author",               # 작성자, None이라면 id 앞부분이 author 역할
            "sha",                  # 현재 repo의 commit SHA, 버전 식별자 역할
            "lastModified",         # 마지막 수정 날짜
            "gated",                # 접근 제한 모델 여부 (동의 절차 필요)
            "disabled",             # 허깅페이스 정책 위반/이슈로 인해 비활성화 상태인지 여부
            "model-index",          # 모델 평가 지표, 벤치마크 관련 데이터
            "config",               # 모델 설정 파일(config.json)에서 읽은 파라미터들
            "cardData",             # 모델 카드(README.md)에서 추출한 메타데이터(YAML 헤더)
            "transformersInfo",     # Transformer 라이브러리 관련 정보(구성, 아키텍쳐 등)
            # "siblings",             # 모델 저장소 파일 리스트(동일한 repo 안의 여러 파일들)
            "spaces",               # 이 모델을 사용하는 허깅페이스 Hugging Face Spaces 정보
            "createdAt",            # 모델 처음 생성 날짜
            "safetensors",          # 모델 저장 포맷이 safetensors인지 여부
            "usedStorage",          # 이거는 뭘까?            
        ]
        self.rename = {
            "_id": "hexid",
            "model-index": "model_index",
            "createdAt": "created_at",
            "lastModified": "last_modified",
            "cardData": "card_data",
            "transformersInfo": "transformers_info",
            "usedStorage": "used_storage"
        }
        self.reorder = [
            "id", "hexid", "sha", "author", "private", "disabled", "gated",
            "pipeline_tag", "library_name", "tags", "likes", "downloads",
            "trending_score", "created_at", "last_modified", "config", "card_data",
            "safetensors", "spaces", "transformers_info", "used_storage", "readme",
            "model_index", "gguf", "inference", "mask_token", "widget_data", "security_repo_status"
        ]
        self.cols = [
            "model_id", "type", "oid", "size", "path",
            "lfs_oid", "lfs_size", "lfs_pointerSize", "xetHash"
        ]
        
    # -----------------
    # ASYNC: aiohttp
    # -----------------    
    async def legacy_reset_csv_async(
        self,
        sort: str = "createdAt",
        limit: Optional[int] = None,
        concurrency: int = 32,
        include_readme: bool = True,
        include_files: bool = True,
        include_community: bool = True,
        batch_size: int = 1_000,
        checkpoint_every_batches: int = 1
    ) -> None:
        if aiohttp is None:
            raise RuntimeError("Async mode requires aiohttp. Install it first: pip install aiohttp")

        # 로깅 시작
        self.logger.info("RESET_CSV(ASYNC) 시작")

        # 사전 처리 모델 확인 후 수집
        done_ids = _load_progress()
        models = self.api.list_models(sort=sort, limit=limit)
        self.logger.info("API list_models 함수 호출 완료")

        # 비동기 처리 세마포어 및 설정 선언
        sema = asyncio.Semaphore(concurrency)
        headers = {
            "User-Agent": "hf-text-scraper/1.0 (+https://github.com/)",
            "Accept": "application/json, text/plain, */*",
        }
        if HF_READ_TOKEN:
            headers["Authorization"] = f"Bearer {HF_READ_TOKEN}"
        cfg = AsyncHTTPConfig()

        # 수집 버퍼 선언
        meta_buf, files_buf, commu_buf = [], [], []

        # 비동기 수집 함수
        async def process(session, model):
            repo_id = getattr(model, "id", None)
            if not repo_id or repo_id in done_ids:
                return None

            async with sema:
                base = {attr: getattr(model, attr) for attr in self.api_schema}
                meta = {}
                readme = {}
                f_list = []
                c_list = []

                async def fetch_with_retry(coro_factory, retries=1):
                    # Session is closed 등 1회 재시도
                    for attempt in range(retries + 1):
                        try:
                            return await coro_factory()
                        except Exception as e:
                            if attempt < retries:
                                # 세션 닫힘 등일 수 있으니 잠깐 쉼
                                await asyncio.sleep(0.5)
                                continue
                            raise

                try:
                    if include_readme or include_files or include_community:
                        meta_full = await fetch_with_retry(lambda: get_model_meta_async(session, repo_id, cfg))
                        if isinstance(meta_full, dict):
                            meta = {k: meta_full.get(k, None) for k in self.req_schema}
                    if include_readme:
                        readme_text = await fetch_with_retry(lambda: get_readme_markdown_async(session, repo_id, cfg))
                        readme = {"readme": readme_text}
                    if not readme:
                        readme = {"readme": None}
                    if include_files:
                        tree = await fetch_with_retry(lambda: get_files_tree_async(session, repo_id, cfg))
                        for ft in tree:
                            f_list.append({"model_id": repo_id} | ft)
                    if include_community:
                        commus = await fetch_with_retry(lambda: get_community_async(session, repo_id, cfg))
                        for c in commus:
                            c_list.append({"model_id": repo_id} | c)

                    merged = base | meta | readme
                    return repo_id, merged, f_list, c_list

                except Exception as e:
                    self.logger.exception(f"모델 {repo_id} 다음과 같은 에러 사유로 인해 스킵합니다. {e}")
                    return None

        async with aiohttp.ClientSession(headers=headers) as session:
            try:
                for bi, batch in enumerate(tqdm(batched(models, batch_size))):
                    filtered = [m for m in batch if getattr(m, "id", None) not in done_ids]
                    if not filtered:
                        continue

                    tasks = {asyncio.create_task(process(session, m)) for m in filtered}

                    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
                        res = await coro
                        if res is None:
                            continue
                        repo_id, meta, f_list, c_list = res
                        meta_buf.append({k: make_hashable(v) for k, v in meta.items()})
                        files_buf.extend(f_list)
                        commu_buf.extend(c_list)
                        done_ids.add(repo_id)

                    # 체크포인트 플러시
                    if (bi + 1) % checkpoint_every_batches == 0:
                        if meta_buf:
                            meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)[self.reorder]
                            _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
                            meta_buf.clear()
                        if files_buf:
                            file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
                            file_df = file_df[file_df["path"].notna()]        # NOT NULL 보호
                            _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
                            files_buf.clear()
                        if commu_buf:
                            commu_df = pd.json_normalize(commu_buf, sep="_")
                            _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])
                            commu_buf.clear()

                        _save_progress(done_ids)

            except Exception as e:
                self.logger.exception(f"배치 처리 중 예외 발생, 부분 저장 후 중단: {e}")

                if meta_buf:
                    meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)[self.reorder]
                    _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
                if files_buf:
                    file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
                    file_df = file_df[file_df["path"].notna()]
                    _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
                if commu_buf:
                    commu_df = pd.json_normalize(commu_buf, sep="_")
                    _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])

                _save_progress(done_ids)
                raise

        # 남은 버퍼 최종 플러시
        if meta_buf:
            meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)[self.reorder]
            _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
        if files_buf:
            file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
            file_df = file_df[file_df["path"].notna()]
            _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
        if commu_buf:
            commu_df = pd.json_normalize(commu_buf, sep="_")
            _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])

        _save_progress(done_ids)
        self.logger.info("RESET_CSV(ASYNC) 완료")
    
    async def reset_csv_async(
        self,
        sort: str = "createdAt",
        limit: Optional[int] = None,
        concurrency: int = 256,              # 동시 태스크 상한(세마포어)
        include_readme: bool = True,
        include_files: bool = True,
        include_community: bool = True,
        batch_size: int = 4096,
        checkpoint_every_batches: int = 1,
        rps: int = 80,                        # 초당 요청수(시작값; 안전선에서 점진 상향 가능)
        timeout_sec: float = 25.0,            # 전체 요청 타임아웃
    ) -> None:
        if aiohttp is None:
            raise RuntimeError("Async mode requires aiohttp. Install it first: pip install aiohttp")

        self.logger.info("RESET_CSV(ASYNC) 시작")

        # 선행 수집된 ID
        done_ids = _load_progress()

        # 모델 목록 조회 (동기 HF SDK)
        models = self.api.list_models(sort=sort, limit=limit)
        self.logger.info("API list_models 함수 호출 완료")

        # 컨트롤 객체들
        sema = asyncio.Semaphore(concurrency)
        limiter = AsyncLimiter(rps, 1)   # 초당 rps개
        headers = {
            "User-Agent": "hf-text-scraper/1.0 (+https://github.com/)",
            "Accept": "application/json, text/plain, */*",
        }
        if HF_READ_TOKEN:
            headers["Authorization"] = f"Bearer {HF_READ_TOKEN}"

        cfg = AsyncHTTPConfig()
        cfg.timeout_sec = timeout_sec  # 헬퍼에서 참조한다고 가정

        # 버퍼
        meta_buf, files_buf, commu_buf = [], [], []

        # -----------------
        # 각 모델 처리 함수
        # -----------------
        async def process(session, model):
            repo_id = getattr(model, "id", None)
            if not repo_id or repo_id in done_ids:
                return None

            async with sema:  # 동시성 상한
                base = {attr: getattr(model, attr, None) for attr in self.api_schema}
                meta = {}
                readme = {}
                f_list = []
                c_list = []

                async def _meta():
                    return await get_model_meta_async(session, repo_id, cfg)

                async def _readme():
                    return await get_readme_markdown_async(session, repo_id, cfg)

                async def _files():
                    return await get_files_tree_async(session, repo_id, cfg)

                async def _commu():
                    return await get_community_async(session, repo_id, cfg)

                try:
                    if include_readme or include_files or include_community:
                        meta_full = await _retry(
                            lambda: _limited_call(limiter, _meta), retries=3
                        )
                        if isinstance(meta_full, dict):
                            meta = {k: meta_full.get(k, None) for k in self.req_schema}

                    if include_readme:
                        readme_text = await _retry(
                            lambda: _limited_call(limiter, _readme), retries=2
                        )
                        readme = {"readme": readme_text}

                    if include_files:
                        tree = await _retry(
                            lambda: _limited_call(limiter, _files), retries=2
                        )
                        for ft in tree:
                            f_list.append({"model_id": repo_id} | ft)

                    if include_community:
                        commus = await _retry(
                            lambda: _limited_call(limiter, _commu), retries=2
                        )
                        for c in commus:
                            c_list.append({"model_id": repo_id} | c)

                    merged = base | meta | readme
                    return repo_id, merged, f_list, c_list

                except Exception as e:
                    # 개별 모델 실패는 로깅만 하고 스킵 (세션 유지)
                    self.logger.error(f"모델 {repo_id} 수집 실패: {e}")
                    return None

        # -----------------
        # 세션 열고 전체 배치 실행
        # -----------------
        connector = _build_connector()
        timeout = _build_timeout(timeout_sec)
        async with aiohttp.ClientSession(headers=headers, timeout=timeout, connector=connector) as session:
            try:
                for bi, batch in enumerate(tqdm(batched(models, batch_size))):
                    filtered = [m for m in batch if getattr(m, "id", None) not in done_ids]
                    if not filtered:
                        continue

                    tasks = [asyncio.create_task(process(session, m)) for m in filtered]

                    # 모든 태스크를 끝까지 기다림: 세션 조기 종료 방지
                    results = await asyncio.gather(*tasks, return_exceptions=True)

                    # 결과 반영
                    for res in results:
                        if isinstance(res, Exception) or res is None:
                            continue
                        repo_id, meta, f_list, c_list = res
                        meta_buf.append({k: make_hashable(v) for k, v in meta.items()})
                        if f_list: files_buf.extend(f_list)
                        if c_list: commu_buf.extend(c_list)
                        done_ids.add(repo_id)

                    # 체크포인트 플러시(매 배치)
                    if (bi + 1) % checkpoint_every_batches == 0:
                        if meta_buf:
                            meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)
                            # 일부 키가 누락될 수 있어 reindex 보호
                            meta_df = meta_df.reindex(columns=self.reorder, fill_value=None)
                            _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
                            meta_buf.clear()

                        if files_buf:
                            file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
                            file_df = file_df[file_df["path"].notna()]  # NOT NULL 보호
                            if not file_df.empty:
                                _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
                            files_buf.clear()

                        if commu_buf:
                            commu_df = pd.json_normalize(commu_buf, sep="_")
                            if not commu_df.empty:
                                _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])
                            commu_buf.clear()

                        _save_progress(done_ids)

            except Exception as e:
                # 배치 루프 도중 예외가 나도 세션을 닫기 전에 버퍼를 최대한 기록
                self.logger.exception(f"배치 처리 중 예외 발생, 부분 저장 후 계속: {e}")

                if meta_buf:
                    meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)
                    meta_df = meta_df.reindex(columns=self.reorder, fill_value=None)
                    _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
                    meta_buf.clear()

                if files_buf:
                    file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
                    file_df = file_df[file_df["path"].notna()]
                    if not file_df.empty:
                        _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
                    files_buf.clear()

                if commu_buf:
                    commu_df = pd.json_normalize(commu_buf, sep="_")
                    if not commu_df.empty:
                        _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])
                    commu_buf.clear()

                _save_progress(done_ids)
                # raise 하지 않음: 세션 스코프 내에서 마무리하고 종료

        # 세션 닫힌 이후에도 남은 버퍼가 있으면 최종 플러시
        if meta_buf:
            meta_df = pd.DataFrame(meta_buf).rename(columns=self.rename)
            meta_df = meta_df.reindex(columns=self.reorder, fill_value=None)
            _append_csv(meta_df, META_CSV, META_SCHEMA, key_cols=["id"])
        if files_buf:
            file_df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
            file_df = file_df[file_df["path"].notna()]
            if not file_df.empty:
                _append_csv(file_df, FILES_CSV, FILES_SCHEMA, key_cols=["model_id", "path"])
        if commu_buf:
            commu_df = pd.json_normalize(commu_buf, sep="_")
            if not commu_df.empty:
                _append_csv(commu_df, COMMU_CSV, COMMU_SCHEMA, key_cols=["model_id", "num"])

        _save_progress(done_ids)
        self.logger.info("RESET_CSV(ASYNC) 완료")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
# =========================
# 1) Sharded CSV append-only writer (no temp big files, auto-rotate)
# =========================
class ShardedCsvWriter:
    """
    base_path.csv -> base_path.csv.shards/base_name-000001.csv[.gz]
    Appends in chunks; rotates when shard_bytes is reached; never rewrites entire file.
    """
    def __init__(
        self,
        base_path: str,
        shard_bytes: int = 512 * 1024 * 1024,  # 512MB per shard
        rows_per_chunk: int = 25_000,
        use_gzip: bool = True,
        encoding: str = "utf-8",
        sep: str = ",",
        lineterminator: str = "\n",
    ):
        self.base_path = base_path
        self.base_dir = os.path.dirname(base_path) or "."
        self.base_name = os.path.basename(base_path).replace(".csv", "").replace(".gz", "")
        self.shard_dir = f"{base_path}.shards"
        self.ext = ".csv.gz" if use_gzip else ".csv"
        self.shard_bytes = shard_bytes
        self.rows_per_chunk = rows_per_chunk
        self.encoding = encoding
        self.sep = sep
        self.lineterminator = lineterminator

        os.makedirs(self.shard_dir, exist_ok=True)
        self.shard_idx = self._detect_last_shard_index()

    def _detect_last_shard_index(self) -> int:
        patt = os.path.join(self.shard_dir, f"{self.base_name}-*.csv*")
        files = sorted(glob.glob(patt))
        if not files:
            return 1
        last = files[-1]
        try:
            num = int(os.path.basename(last).split("-")[-1].split(".")[0])
            # If last shard is nearly full, start a new one
            if os.path.getsize(last) >= int(self.shard_bytes * 0.98):
                return num + 1
            return num
        except Exception:
            return 1

    def _cur_path(self) -> str:
        return os.path.join(self.shard_dir, f"{self.base_name}-{self.shard_idx:06d}{self.ext}")

    def _rotate_if_needed(self):
        p = self._cur_path()
        if os.path.exists(p) and os.path.getsize(p) >= self.shard_bytes:
            self.shard_idx += 1

    def _open_append(self, path: str):
        if self.ext.endswith(".gz"):
            return gzip.open(path, mode="at", encoding=self.encoding, newline="")
        return open(path, mode="a", encoding=self.encoding, newline="")

    def write_df(self, df: pd.DataFrame):
        if df is None or len(df) == 0:
            return
        # write in chunks to keep memory profile stable
        for start in range(0, len(df), self.rows_per_chunk):
            chunk = df.iloc[start:start + self.rows_per_chunk]
            self._rotate_if_needed()
            path = self._cur_path()
            write_header = not os.path.exists(path) or os.path.getsize(path) == 0
            with self._open_append(path) as fh:
                chunk.to_csv(
                    fh,
                    index=False,
                    sep=self.sep,
                    escapechar="\\",
                    header=write_header,
                    lineterminator=self.lineterminator,
                )

# =========================
# 2) SQLite-based dedup index (to avoid cross-shard duplicates)
# =========================
class DedupIndex:
    """
    Lightweight SQLite index to prevent duplicate rows across shards.
    Composite keys are joined with '\x1f'.
    """
    def __init__(self, db_path: str, table_name: str, key_cols: Sequence[str]):
        self.db_path = db_path
        self.table = table_name
        self.key_cols = list(key_cols)
        os.makedirs(os.path.dirname(db_path) or ".", exist_ok=True)
        self.conn = sqlite3.connect(self.db_path, isolation_level=None)  # autocommit
        self.conn.execute("PRAGMA journal_mode=WAL;")
        self.conn.execute("PRAGMA synchronous=NORMAL;")
        self.conn.execute(f"CREATE TABLE IF NOT EXISTS {self.table} (k TEXT PRIMARY KEY)")
        self._ins_sql = f"INSERT OR IGNORE INTO {self.table}(k) VALUES (?)"

    @staticmethod
    def _make_keys_from_df(df: pd.DataFrame, key_cols: Sequence[str]) -> List[str]:
        if len(key_cols) == 1:
            return df[key_cols[0]].astype(str).tolist()
        return (df[key_cols].astype(str).agg("\x1f".join, axis=1)).tolist()

    def mask_new(self, df: pd.DataFrame) -> pd.Series:
        if len(df) == 0:
            return pd.Series([], dtype=bool)
        keys = self._make_keys_from_df(df, self.key_cols)
        mask = pd.Series(True, index=df.index)
        # chunked IN query
        cur = self.conn.cursor()
        existing = set()
        for i in range(0, len(keys), 10_000):
            part = keys[i:i+10_000]
            if not part:
                continue
            q_marks = ",".join("?" for _ in part)
            for (k,) in cur.execute(f"SELECT k FROM {self.table} WHERE k IN ({q_marks})", part):
                existing.add(k)
        for i, k in zip(df.index, keys):
            if k in existing:
                mask.at[i] = False
        return mask

    def insert_df(self, df: pd.DataFrame):
        if len(df) == 0:
            return
        keys = self._make_keys_from_df(df, self.key_cols)
        self.conn.executemany(self._ins_sql, [(k,) for k in keys])

    def close(self):
        try:
            self.conn.close()
        except Exception:
            pass

# =========================
# 3) Progress store (SQLite set instead of giant JSON list)
# =========================
class ProgressStore:
    def __init__(self, db_path: str = "progress.db"):
        os.makedirs(os.path.dirname(db_path) or ".", exist_ok=True)
        self.conn = sqlite3.connect(db_path, isolation_level=None)  # autocommit
        self.conn.execute("PRAGMA journal_mode=WAL;")
        self.conn.execute("PRAGMA synchronous=NORMAL;")
        self.conn.execute("CREATE TABLE IF NOT EXISTS done_ids (id TEXT PRIMARY KEY)")
        self._ins_sql = "INSERT OR IGNORE INTO done_ids(id) VALUES (?)"

    def contains(self, repo_id: str) -> bool:
        cur = self.conn.cursor()
        row = cur.execute("SELECT 1 FROM done_ids WHERE id=?", (repo_id,)).fetchone()
        return row is not None

    def add(self, repo_id: str):
        self.conn.execute(self._ins_sql, (repo_id,))

    def add_many(self, repo_ids: Iterable[str]):
        self.conn.executemany(self._ins_sql, ((r,) for r in repo_ids))

    def close(self):
        try:
            self.conn.close()
        except Exception:
            pass

# =========================
# 4) Simple token-bucket RPS limiter (no external dependency)
# =========================
class SimpleRateLimiter:
    def __init__(self, rps: float):
        self.rate = max(0.1, float(rps))     # 초당 토큰 보충 속도
        self.capacity = self.rate            # 1초치 버스트 허용
        self.tokens = self.capacity
        self.last = time.monotonic()
        self._lock = asyncio.Lock()

    async def acquire(self) -> None:
        while True:
            async with self._lock:
                now = time.monotonic()
                delta = now - self.last
                # 경과한 시간만큼 토큰 보충
                self.tokens = min(self.capacity, self.tokens + delta * self.rate)
                self.last = now
                if self.tokens >= 1.0:
                    self.tokens -= 1.0
                    return
                # 다음 토큰까지 대기할 시간 계산
                wait = (1.0 - self.tokens) / self.rate
            await asyncio.sleep(max(wait, 0.005))

# =========================
# 5) HuggingFace class (updated)
# =========================
class HuggingFace:
    def __init__(self) -> None:
        self.api = HfApi()
        self.logger = logging.getLogger(self.__class__.__name__)
        # build_session() is assumed to exist in your environment
        try:
            self.session = build_session()  # type: ignore
        except Exception:
            self.session = None
        self.base_url = "https://huggingface.co/"

        self.api_schema = [
            "id",                   # 모델 전체 ID, author/model_name의 형태
            "gguf",                 # GGUF 포맷 변환 여부
            "inference",            # Inference API 지원 여부
            "mask_token",           # BERT류 마스킹 토큰
            "widget_data",          # 위젯 예시 입출력
            "trending_score",       # 인기도 점수
            "security_repo_status", # 보안 관련 검토 상태
        ]
        self.req_schema = [
            "_id", "id", "private", "pipeline_tag", "library_name", "tags",
            "downloads", "likes", "modelId", "author", "sha", "lastModified",
            "gated", "disabled", "model-index", "config", "cardData",
            "transformersInfo", "spaces", "createdAt", "safetensors", "usedStorage",
        ]
        self.rename = {
            "_id": "hexid",
            "model-index": "model_index",
            "createdAt": "created_at",
            "lastModified": "last_modified",
            "cardData": "card_data",
            "transformersInfo": "transformers_info",
            "usedStorage": "used_storage",
        }
        self.reorder = [
            "id", "hexid", "sha", "author", "private", "disabled", "gated",
            "pipeline_tag", "library_name", "tags", "likes", "downloads",
            "trending_score", "created_at", "last_modified", "config", "card_data",
            "safetensors", "spaces", "transformers_info", "used_storage", "readme",
            "model_index", "gguf", "inference", "mask_token", "widget_data", "security_repo_status",
        ]
        self.cols = [
            "model_id", "type", "oid", "size", "path",
            "lfs_oid", "lfs_size", "lfs_pointerSize", "xetHash",
        ]

    # ------------------------------------------------------------------
    # Legacy entrypoint -> forward to new one (prevents old CSV rewrite)
    # ------------------------------------------------------------------
    async def legacy_reset_csv_async(
        self,
        sort: str = "createdAt",
        limit: Optional[int] = None,
        concurrency: int = 32,
        include_readme: bool = True,
        include_files: bool = True,
        include_community: bool = True,
        batch_size: int = 1_000,
        checkpoint_every_batches: int = 1,
    ) -> None:
        self.logger.info("legacy_reset_csv_async is deprecated; forwarding to reset_csv_async")
        await self.reset_csv_async(
            sort=sort,
            limit=limit,
            concurrency=concurrency,
            include_readme=include_readme,
            include_files=include_files,
            include_community=include_community,
            batch_size=batch_size,
            checkpoint_every_batches=checkpoint_every_batches,
        )

    # ------------------------------------------------------------------
    # New implementation: shard writers + sqlite indexes + progress DB
    # ------------------------------------------------------------------
    async def reset_csv_async(
        self,
        sort: str = "createdAt",
        limit: Optional[int] = None,
        concurrency: int = 128,              # 동시 태스크 상한
        include_readme: bool = True,
        include_files: bool = True,
        include_community: bool = True,
        batch_size: int = 2048,
        checkpoint_every_batches: int = 1,
        rps: int = 100,                       # 초당 요청수 제한 (간단 버킷)
        timeout_sec: float = 25.0,            # 요청 타임아웃(소켓)
        shard_bytes: int = 512 * 1024 * 1024, # 샤드 회전 크기
        rows_per_chunk: int = 25_000,         # CSV 쓰기 청크
        use_gzip: bool = True,                # CSV.gz로 저장
        progress_db_path: str = "progress.db",
        dedup_db_path: str = "dedup-index.db",
    ) -> None:
        if aiohttp is None:
            raise RuntimeError("Async mode requires aiohttp. Install it first: pip install aiohttp")

        self.logger.info("RESET_CSV(ASYNC) 시작")

        # ---------- writers / indexes / progress ----------
        meta_writer  = ShardedCsvWriter(META_CSV,  shard_bytes=shard_bytes, rows_per_chunk=rows_per_chunk, use_gzip=use_gzip, encoding=ENCODING_TYPE)
        files_writer = ShardedCsvWriter(FILES_CSV, shard_bytes=shard_bytes, rows_per_chunk=rows_per_chunk, use_gzip=use_gzip, encoding=ENCODING_TYPE)
        commu_writer = ShardedCsvWriter(COMMU_CSV, shard_bytes=shard_bytes, rows_per_chunk=rows_per_chunk, use_gzip=use_gzip, encoding=ENCODING_TYPE)

        meta_idx  = DedupIndex(dedup_db_path, "meta_idx",  ["id"])
        files_idx = DedupIndex(dedup_db_path, "files_idx", ["model_id", "path"])
        commu_idx = DedupIndex(dedup_db_path, "commu_idx", ["model_id", "num"])

        progress = ProgressStore(progress_db_path)

        # ---------- already done set is DB-backed ----------
        def _already_done(repo_id: str) -> bool:
            return progress.contains(repo_id)

        # ---------- models list ----------
        models = self.api.list_models(sort=sort, limit=limit)
        self.logger.info("API list_models 함수 호출 완료")

        # ---------- concurrency / rps / aiohttp ----------
        sema = asyncio.Semaphore(concurrency)
        limiter = SimpleRateLimiter(rps=rps)

        headers = {
            "User-Agent": "hf-text-scraper/1.0 (+https://github.com/)",
            "Accept": "application/json, text/plain, */*",
        }
        try:
            if HF_READ_TOKEN:  # type: ignore
                headers["Authorization"] = f"Bearer {HF_READ_TOKEN}"  # type: ignore
        except NameError:
            pass

        timeout = aiohttp.ClientTimeout(total=None, sock_connect=timeout_sec, sock_read=timeout_sec)
        connector = aiohttp.TCPConnector(limit=concurrency * 2, ttl_dns_cache=300, enable_cleanup_closed=True)

        # ---------- buffers ----------
        meta_buf: List[Dict[str, Any]] = []
        files_buf: List[Dict[str, Any]] = []
        commu_buf: List[Dict[str, Any]] = []

        # ---------- helpers ----------
        async def _limited_call(coro_factory):
            await limiter.acquire()
            return await coro_factory()

        async def _retry(coro_factory, retries=2, base_delay=0.5):
            err = None
            for attempt in range(retries + 1):
                try:
                    return await _limited_call(coro_factory)
                except Exception as e:
                    err = e
                    if attempt < retries:
                        await asyncio.sleep(base_delay * (2 ** attempt))
                        continue
                    raise err

        def _flush_meta():
            nonlocal meta_buf
            if not meta_buf:
                return
            df = pd.DataFrame(meta_buf).rename(columns=self.rename)
            # ensure all columns exist
            for c in self.reorder:
                if c not in df.columns:
                    df[c] = pd.NA
            df = df[self.reorder]
            # dedup + write
            mask = meta_idx.mask_new(df[["id"]])
            new_df = df[mask]
            if len(new_df):
                meta_writer.write_df(new_df)
                meta_idx.insert_df(new_df)
            meta_buf = []

        def _flush_files():
            nonlocal files_buf
            if not files_buf:
                return
            df = pd.json_normalize(files_buf, sep="_").reindex(columns=self.cols)
            df = df[df["path"].notna()]
            if df.empty:
                files_buf = []
                return
            mask = files_idx.mask_new(df[["model_id", "path"]])
            new_df = df[mask]
            if len(new_df):
                files_writer.write_df(new_df)
                files_idx.insert_df(new_df)
            files_buf = []

        def _flush_commu():
            nonlocal commu_buf
            if not commu_buf:
                return
            df = pd.json_normalize(commu_buf, sep="_")
            # ensure keys
            if "model_id" not in df.columns:
                df["model_id"] = pd.NA
            if "num" not in df.columns:
                df["num"] = pd.NA
            mask = commu_idx.mask_new(df[["model_id", "num"]])
            new_df = df[mask]
            if len(new_df):
                commu_writer.write_df(new_df)
                commu_idx.insert_df(new_df)
            commu_buf = []

        # -----------------
        # per-model task
        # -----------------
        async def process(session, model):
            repo_id = getattr(model, "id", None)
            if not repo_id or _already_done(repo_id):
                return None

            async with sema:
                base = {attr: getattr(model, attr, None) for attr in self.api_schema}
                meta: Dict[str, Any] = {}
                readme: Dict[str, Any] = {}
                f_list: List[Dict[str, Any]] = []
                c_list: List[Dict[str, Any]] = []

                # these functions are assumed to exist in your environment
                async def _meta():
                    return await get_model_meta_async(session, repo_id, AsyncHTTPConfig())  # type: ignore

                async def _readme():
                    return await get_readme_markdown_async(session, repo_id, AsyncHTTPConfig())  # type: ignore

                async def _files():
                    return await get_files_tree_async(session, repo_id, AsyncHTTPConfig())  # type: ignore

                async def _commu():
                    return await get_community_async(session, repo_id, AsyncHTTPConfig())  # type: ignore

                try:
                    if include_readme or include_files or include_community:
                        meta_full = await _retry(_meta, retries=3)
                        if isinstance(meta_full, dict):
                            meta = {k: meta_full.get(k, None) for k in self.req_schema}

                    if include_readme:
                        readme_text = await _retry(_readme, retries=2)
                        readme = {"readme": readme_text}
                    else:
                        readme = {"readme": None}

                    if include_files:
                        tree = await _retry(_files, retries=2)
                        for ft in tree:
                            f_list.append({"model_id": repo_id} | ft)

                    if include_community:
                        commus = await _retry(_commu, retries=2)
                        for c in commus:
                            c_list.append({"model_id": repo_id} | c)

                    merged = base | meta | readme
                    return repo_id, merged, f_list, c_list

                except Exception as e:
                    self.logger.error(f"모델 {repo_id} 수집 실패: {e}")
                    return None

        # -----------------
        # run
        # -----------------
        async with aiohttp.ClientSession(headers=headers, timeout=timeout, connector=connector) as session:
            try:
                for bi, batch in enumerate(tqdm(batched(models, batch_size))):
                    filtered = [m for m in batch if getattr(m, "id", None) and not _already_done(getattr(m, "id"))]
                    if not filtered:
                        continue

                    tasks = [asyncio.create_task(process(session, m)) for m in filtered]
                    results = await asyncio.gather(*tasks, return_exceptions=True)

                    for res in results:
                        if isinstance(res, Exception) or res is None:
                            continue
                        repo_id, meta, f_list, c_list = res
                        meta_buf.append({k: make_hashable(v) for k, v in meta.items()})
                        if f_list: files_buf.extend(f_list)
                        if c_list: commu_buf.extend(c_list)
                        # mark progress ASAP (after assembling buffers)
                        progress.add(repo_id)

                    # checkpoint flush every batch (or based on your cadence)
                    if (bi + 1) % checkpoint_every_batches == 0:
                        _flush_meta()
                        _flush_files()
                        _flush_commu()

            except Exception as e:
                # flush what we have, then continue to close gracefully
                self.logger.exception(f"배치 처리 중 예외 발생, 부분 저장: {e}")
                _flush_meta()
                _flush_files()
                _flush_commu()

        # final flush
        _flush_meta()
        _flush_files()
        _flush_commu()

        # close DBs
        try:
            meta_idx.close(); files_idx.close(); commu_idx.close(); progress.close()
        except Exception:
            pass

        self.logger.info("RESET_CSV(ASYNC) 완료")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 8. Main

In [0]:
setup_logging()

hf = HuggingFace()

await hf.legacy_reset_csv_async(
    sort="trendingScore",
    limit=None,
    concurrency=256,
    include_readme=False,
    include_files=True,
    include_community=True,
    batch_size=2048,
    checkpoint_every_batches=1,
    # rps=120,
    # timeout_sec=25.0
)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 9. Save Catalog for Delta Table

## 9-1. Create Schema(Database)

In [0]:
# DATABASE_NAME = "huggingface"

# spark.sql(f"CREATE SCHEMA IF NOT EXISTS hive_metastore.{DATABASE_NAME}")
# spark.sql("SHOW DATABASES").show()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 9-2. Save Table

In [0]:
# df_list = [
#   pd.read_csv("hf_metadata.csv", encoding=ENCODING_TYPE),
#   pd.read_csv("hf_files.csv", encoding=ENCODING_TYPE),
#   pd.read_csv("hf_community.csv", encoding=ENCODING_TYPE)
# ]

# for df, tb_nm in zip(df_list, ["hf_meta", "hf_files", "hf_community"]):
#   if isinstance(df, pd.DataFrame):
#     df = spark.createDataFrame(df)
  
#   (
#     df
#     .write
#     .format("delta")
#     .mode("overwrite")
#     # catalog.schema(database).table
#     .saveAsTable(f"hive_metastore.{DATABASE_NAME}.{tb_nm}")
#   )

# print(f"Delta Table 저장 완료")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 9-3. Delete Schema, Table

In [0]:
# DEL_DB_NAME = "huggingface"
# spark.sql(f"DROP SCHEMA hive_metastore.{DEL_DB_NAME} CASCADE")
# spark.sql("SHOW DATABASES").show()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 10. Validation

In [0]:
pd.read_csv("hf_metadata.csv", encoding=ENCODING_TYPE).head()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
pd.read_csv("hf_files.csv", encoding=ENCODING_TYPE).head()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
pd.read_csv("hf_community.csv", encoding=ENCODING_TYPE).head()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 11. Save PostgreSQL Table (Azure VM)

## 11-1. Install and Import Libraries

In [0]:
%sh
pip install --quiet psycopg2-binary

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
from typing import Union
import pandas as pd
import psycopg2
import pyspark
from pyspark import sql as pysql
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    StringType, ShortType, IntegerType, LongType, DoubleType, BooleanType,
    TimestampType, ArrayType, MapType
)
from pyspark.sql import functions as F

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-2. Show Database

In [ ]:
global_host = "YOUR_HOST"
global_port = "YOUR_PORT"
global_user = "YOUR_USER"
global_password = "YOUR_PASSWORD"

In [0]:
def show_databases() -> None:
    host = global_host
    port = global_port
    user = global_user
    password = global_password

    jdbc_url = f"jdbc:postgresql://{host}:{port}/postgres?user={user}&password={password}"
    query = "(SELECT datname FROM pg_database WHERE datistemplate = false) AS databases"
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-3. Show Schema

In [0]:
def show_schemas(database: str = "postgres") -> None:
    host = global_host
    port = global_port
    user = global_user
    password = global_password

    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}?user={user}&password={password}"
    query = "(SELECT schema_name FROM information_schema.schemata) AS schemas"
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-4. Show Table

In [0]:
def show_tables(schema: str, database: str = "postgres") -> None:
    host = global_host
    port = global_port
    user = global_user
    password = global_password

    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}?user={user}&password={password}"
    query = f"""
        (SELECT table_name
         FROM information_schema.tables
         WHERE table_type='BASE TABLE'
           AND table_schema = '{schema}'
         ORDER BY table_name) AS tables
    """
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-5. Show Data (in specific table)

In [0]:
def show_datas(schema: str, table: str, database: str = "postgres") -> None:
    host = global_host
    port = global_port
    user = global_user
    password = global_password

    # DB 접속 URL
    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"

    # 스키마.테이블 지정
    dbtable = f"{schema}.{table}"
    df = (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("user", user)
        .option("password", password)
        .option("driver", "org.postgresql.Driver")
        .option("dbtable", dbtable)
        .load()
    )
    df.show(truncate=False)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-6. Set Schema Using Pyspark StructType

### 1. Metadata

In [0]:
meta_schema = StructType([
    StructField("id",                   StringType(),       False), # PK, NOT NULL
    StructField("hexid",                StringType(),       True),  # CHAR(24)
    StructField("sha",                  StringType(),       True),  # CHAR(40)
    StructField("author",               StringType(),       True),  # 
    StructField("private",              BooleanType(),      True),  # 
    StructField("disabled",             BooleanType(),      True),  # 
    StructField("gated",                StringType(),       True),  #   
    StructField("pipeline_tag",         StringType(),       True),  # 
    StructField("library_name",         StringType(),       True),  # 
    StructField("tags",                 StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("likes",                IntegerType(),      True),  # 
    StructField("downloads",            IntegerType(),      True),  # 
    StructField("trending_score",       DoubleType(),       False), # NOT NULL 
    StructField("created_at",           TimestampType(),    True),  # 
    StructField("last_modified",        TimestampType(),    True),  # 
    StructField("config",               StringType(),       True),  # JSON 문자열
    StructField("card_data",            StringType(),       True),  # JSON 문자열
    StructField("safetensors",          StringType(),       True),  # JSON 문자열
    StructField("spaces",               StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("transformers_info",    StringType(),       True),  # JSON 문자열
    StructField("used_storage",         LongType(),         True),  # 
    StructField("readme",               StringType(),       True),  # 

    StructField("model_index",          StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("gguf",                 StringType(),       True),  # 
    StructField("inference",            StringType(),       True),  # 
    StructField("mask_token",           StringType(),       True),  # 
    StructField("widget_data",          StringType(),       True),  # 
    StructField("security_repo_status", StringType(),       True)   # 
])

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 2. Files

In [0]:
file_schema = StructType([
    StructField("model_id",        StringType(),    False), # FK, NOT NULL
    StructField("type",            StringType(),    False), # NOT NULL
    StructField("oid",             StringType(),    False), # NOT NULL (CHAR(40))
    StructField("size",            LongType(),      False), # NOT NULL
    StructField("path",            StringType(),    True),  # 
    StructField("lfs_oid",         StringType(),    True),  # CHAR(64)
    StructField("lfs_size",        LongType(),      True),  # 
    StructField("lfs_pointerSize", IntegerType(),   True),  # 
    StructField("xetHash",         StringType(),    True)   # CHAR(64)
])

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 3. Community

In [0]:
comu_schema = StructType([
    StructField("model_id",                     StringType(),       False), # FK, NOT NULL
    StructField("num",                          IntegerType(),      False), # PK*, NOT NULL
    StructField("title",                        StringType(),       True),  # 
    StructField("status",                       StringType(),       True),  # 
    StructField("createdAt",                    TimestampType(),    True),  # 
    StructField("isPullRequest",                BooleanType(),      True),  # 
    StructField("numComments",                  IntegerType(),      True),  # 
    StructField("topReactions",                 StringType(),       True),  # JSON 문자열
    StructField("numReactionUsers",             IntegerType(),      True),  # 
    StructField("pinned",                       BooleanType(),      True),  # 
    StructField("author__id",                   StringType(),       True),  # CHAR(24)
    StructField("author_avatarUrl",             StringType(),       True),  # 
    StructField("author_fullname",              StringType(),       True),  # 
    StructField("author_name",                  StringType(),       True),  # 
    StructField("author_type",                  StringType(),       True),  # 
    StructField("author_isPro",                 BooleanType(),      False), # 
    StructField("author_isHf",                  BooleanType(),      False), # 
    StructField("author_isHfAdmin",             BooleanType(),      False), # 
    StructField("author_isMod",                 BooleanType(),      False), # 
    StructField("repo_name",                    StringType(),       False), # PK*, NOT NULL
    StructField("repo_type",                    StringType(),       True),  # 
    StructField("repoOwner_name",               StringType(),       True),  # 
    StructField("repoOwner_isParticipating",    BooleanType(),      True),  # 
    StructField("repoOwner_type",               StringType(),       True),  # 
    StructField("repoOwner_isDiscussionAuthor", BooleanType(),      True),  # 
    StructField("author_followerCount",         DoubleType(),       True),  # 
])

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-7. Create PostgreSQL Schema

In [0]:
SCHEMA_LEVEL = "bronze"
META_TABLE_NAME = "brz_hf_meta"
FILE_TABLE_NAME = "brz_hf_files"
COMU_TABLE_NAME = "brz_hf_community"

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
conn = psycopg2.connect(
    host=global_host,
    port=global_port,
    dbname="postgres",
    user=global_user,
    password=global_password
)
cur = conn.cursor()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 0. DROP CASCADE

In [0]:
# 기존 테이블 드롭 (CASCADE 포함)
cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_LEVEL}.{META_TABLE_NAME} CASCADE;")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 1. Metadata

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{META_TABLE_NAME} (
  id                   varchar(255) NOT NULL PRIMARY KEY,
  hexid                char(24),
  sha                  char(40),
  author               varchar(100),
  private              boolean,
  disabled             boolean,
  gated                varchar(6),
  pipeline_tag         varchar(100),
  library_name         varchar(100),
  tags                 text ,
  likes                 double precision,
  downloads             double precision,
  trending_score        double precision,
  created_at           timestamptz,
  last_modified        timestamptz,
  config                text,
  card_data             text,
  safetensors           text,
  spaces                text,
  transformers_info     text,
  used_storage          double precision,
  readme               text,
  model_index          text ,
  gguf                 text,
  inference            text,
  mask_token           text,
  widget_data          text,
  security_repo_status text
);
""")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 2. Files

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{FILE_TABLE_NAME} (
    model_id        VARCHAR(255) NOT NULL,
    type            VARCHAR(50)  NOT NULL,
    oid             CHAR(40)     NOT NULL,
    size            BIGINT,
    path            VARCHAR(255),
    lfs_oid         CHAR(64),
    lfs_size        DOUBLE PRECISION,
    lfs_pointerSize DOUBLE PRECISION,
    xetHash         CHAR(64),

    -- PK 정의 (복합 기본키)
    CONSTRAINT pk_files PRIMARY KEY (model_id, path),
    -- FK 정의 (모델 메타 테이블과 연결)
    CONSTRAINT fk_hfmeta
        FOREIGN KEY (model_id)
        REFERENCES {SCHEMA_LEVEL}.{META_TABLE_NAME} (id)
);
""")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 3. Community

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{COMU_TABLE_NAME} (
    model_id                    VARCHAR(255) NOT NULL,
    num                         INTEGER NOT NULL,
    title                        text,
    status                      VARCHAR(20),
    createdAt                   TIMESTAMPTZ,
    isPullRequest               BOOLEAN,
    numComments                 INTEGER,
    topReactions                 text,
    numReactionUsers            INTEGER,
    pinned                      BOOLEAN,
    author__id                  CHAR(24),
    author_avatarUrl            VARCHAR(255),
    author_fullname             VARCHAR(200),
    author_name                 VARCHAR(100),
    author_type                 VARCHAR(20),
    author_isPro                BOOLEAN,
    author_isHf                 BOOLEAN,
    author_isHfAdmin            BOOLEAN,
    author_isMod                BOOLEAN,
    repo_name                   VARCHAR(255) NOT NULL,
    repo_type                   VARCHAR(20),
    repoOwner_name              VARCHAR(100),
    repoOwner_isParticipating   BOOLEAN,
    repoOwner_type              VARCHAR(20),
    repoOwner_isDiscussionAuthor BOOLEAN,
    author_followerCount        DOUBLE PRECISION,

    -- PK 정의 (복합 기본키)
    CONSTRAINT pk_community PRIMARY KEY (num, repo_name),
    -- FK 정의 (모델 메타 테이블과 연결)
    CONSTRAINT fk_hfmeta
        FOREIGN KEY (model_id)
        REFERENCES {SCHEMA_LEVEL}.{META_TABLE_NAME} (id)
);
""")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

In [0]:
conn.commit()
cur.close()
conn.close()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-8. UDF Example

In [0]:
# # 데이터베이스 목록을 조회하는 코드
# show_databases()

# # 스키마 목록을 조회하는 코드
# show_schemas()

# # 특정 스키마 아래의 테이블 목록을 조회하는 코드
# show_tables(schema="test_data")

# # 특정 테이블의 데이터를 조회하는 코드
# show_datas(schema="test_data", table="test_table")

# # 특정 테이블에 데이터를 삽입하는 코드
# # mode 종류: overwrite, append, ignore, error
# insert_data(data=test_df, schema="test_data", table="test_table", mode="overwrite")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

## 11-9. INSERT data

In [0]:
def insert_data(
    data: Union[pd.DataFrame, pyspark.sql.DataFrame],
    schema: str,
    table: str,
    mode: str = "overwrite"
) -> None:
    # 데이터프레임 생성
    # spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()
    columns = list(data.columns)
    if isinstance(data, pd.DataFrame):
        # data = spark.createDataFrame(data, schema=columns)
        data = spark.createDataFrame(data)
    
    # 인증 정보 하드코딩
    jdbc_user = global_user
    jdbc_pass = global_password

    # 연결 문자열 하드코딩
    host = global_host
    port = global_port
    jdbc_url = f"jdbc:postgresql://{host}:{port}/postgres"

    data.write.format("jdbc").mode(mode).options(
        url=jdbc_url,
        user=jdbc_user,
        password=jdbc_pass,
        dbtable=f"{schema}.{table.lower()}",
        # overwrite일 때 DROP 후 CREATE가 아니라, TRUNCATE TABLE로 데이터만 비움
        truncate=True,
        # 대량 적재 튜닝: INSERT할 때 한 번에 몇 행씩 묶어서 넣을지
        batchsize=1000,
        # 스키마 적재 튜닝: INSERT시 JDBC는 setString() 함수로 text 자동 매핑
        # 이 옵션을 사용하면 unspecified로 넘겨 PostgreSQL이 자동 캐스팅
        # stringtype="unspecified"
    ).save()

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 1. Metadata

In [0]:
tmp_df = pd.read_csv('hf_metadata.csv', encoding=ENCODING_TYPE)
tmp_df["created_at"] = pd.to_datetime(tmp_df["created_at"], utc=True, errors="coerce")
tmp_df["last_modified"] = pd.to_datetime(tmp_df["last_modified"], utc=True, errors="coerce")

# 특정 테이블에 데이터를 삽입하는 코드
# mode 종류: overwrite, append, ignore, error
insert_data(
    data=spark.createDataFrame(tmp_df, schema=meta_schema),
    schema=SCHEMA_LEVEL,
    table=META_TABLE_NAME,
    mode="overwrite"
)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 2. Files

In [0]:
tmp_df = pd.read_csv('hf_files.csv', encoding=ENCODING_TYPE)
tmp_df = tmp_df[tmp_df['path'].notna()]

# 특정 테이블에 데이터를 삽입하는 코드
# mode 종류: overwrite, append, ignore, error
insert_data(
    data=spark.createDataFrame(tmp_df, schema=file_schema),
    schema=SCHEMA_LEVEL,
    table=FILE_TABLE_NAME,
    mode="overwrite"
)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

### 3. Community

In [0]:
tmp_df = pd.read_csv('hf_community.csv', encoding=ENCODING_TYPE)
tmp_df["createdAt"] = pd.to_datetime(tmp_df["createdAt"], utc=True, errors="coerce")

# 특정 테이블에 데이터를 삽입하는 코드
# mode 종류: overwrite, append, ignore, error
insert_data(
    data=spark.createDataFrame(tmp_df, schema=comu_schema),
    schema=SCHEMA_LEVEL,
    table=COMU_TABLE_NAME,
    mode="overwrite"
)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'

# 12. END...?

In [0]:
df = pd.read_csv('hf_metadata.csv', encoding='utf-8-sig')
df.schema

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4884005352694818>, line 21
     18 from typing import Any, Optional, Union, Iterable, Sequence, List, Dict
     19 from urllib.parse import urlparse
---> 21 import aiohttp
     22 from aiolimiter import AsyncLimiter
     23 from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'aiohttp'